user_proj_repo and user_proj_repo_after_6mon from GraphQL API include repos of hackathon projects, which should be removed.

In [ ]:
"""
Update projects table to keep repo names in a same form:
ALTER TABLE projects
ADD COLUMN github_repos text[];

UPDATE projects
SET github_repos = ARRAY(
  SELECT regexp_replace(link, '^https?://github\.com/', '')
  FROM unnest(repo_links) AS link
)
WHERE repo_links IS NOT NULL;

"""

"""
SELECT
  project_id,
  repo_links AS original_links,
  github_repos AS cleaned_links
FROM projects
LIMIT 10;
"""

"""
UPDATE projects
SET github_repos = ARRAY(
  SELECT regexp_replace(
           regexp_replace(link, '^https?://github\.com/', ''), -- 去掉前缀
           '^([^/]+/[^/]+).*', '\1'                           -- 保留前两部分
         )
  FROM unnest(repo_links) AS link
)
WHERE repo_links IS NOT NULL;
"""

# Remove duplicates
"""
UPDATE projects
SET github_repos = ARRAY(
  SELECT DISTINCT repo
  FROM unnest(github_repos) AS repo
)
WHERE github_repos IS NOT NULL;

"""

In [ ]:
"""
ALTER TABLE user_proj_repo 
ADD COLUMN repos_outside jsonb;

ALTER TABLE user_proj_repo_after_6mon
ADD COLUMN repos_outside jsonb;
"""

In [ ]:
"""
UPDATE user_proj_repo upr
SET repos_outside = subq.repos_outside
FROM (
    SELECT 
        upr2.user_id,
        upr2.project_id,
        upr2.window_type,
        COALESCE(
            jsonb_agg(repo_elem) FILTER (WHERE NOT repo_elem = ANY(p.github_repos)),
            '[]'::jsonb
        ) AS repos_outside
    FROM user_proj_repo upr2
    INNER JOIN projects p ON p.project_id = upr2.project_id
    CROSS JOIN LATERAL jsonb_array_elements_text(upr2.repos) AS repo_elem
    GROUP BY upr2.user_id, upr2.project_id, upr2.window_type
) subq
WHERE upr.user_id = subq.user_id
  AND upr.project_id = subq.project_id
  AND upr.window_type = subq.window_type
  AND upr.repos_outside IS NULL;


UPDATE user_proj_repo_after_6mon upr
SET repos_outside = subq.repos_outside
FROM (
    SELECT 
        upr2.user_id,
        upr2.project_id,
        upr2.window_type,
        COALESCE(
            jsonb_agg(repo_elem) FILTER (WHERE NOT repo_elem = ANY(p.github_repos)),
            '[]'::jsonb
        ) AS repos_outside
    FROM user_proj_repo_after_6mon upr2
    INNER JOIN projects p ON p.project_id = upr2.project_id
    CROSS JOIN LATERAL jsonb_array_elements_text(upr2.repos) AS repo_elem
    GROUP BY upr2.user_id, upr2.project_id, upr2.window_type
) subq
WHERE upr.user_id = subq.user_id
  AND upr.project_id = subq.project_id
  AND upr.window_type = subq.window_type
  AND upr.repos_outside IS NULL;
"""